In [1]:
import cv2
from pathlib import Path
from datetime import datetime

# Haar cascades (make sure these files are in the same folder as this notebook)
FACE_CASCADE_PATH = Path("haarcascade_frontalface_default.xml")
EYE_CASCADE_PATH  = Path("haarcascade_eye.xml")

# Output dataset directories (auto-created)
DATASET_DIR = Path("dataset")
FACE_OUT    = DATASET_DIR / "faces"
EYE_OUT     = DATASET_DIR / "eyes"
FACE_OUT.mkdir(parents=True, exist_ok=True)
EYE_OUT.mkdir(parents=True, exist_ok=True)

print("Face cascade exists:", FACE_CASCADE_PATH.exists())
print("Eye cascade  exists:", EYE_CASCADE_PATH.exists())
print("Dataset folders:", FACE_OUT.resolve(), "|", EYE_OUT.resolve())


Face cascade exists: True
Eye cascade  exists: True
Dataset folders: /Users/jaahanavajoshi/Documents/GitHub/Talha/dataset/faces | /Users/jaahanavajoshi/Documents/GitHub/Talha/dataset/eyes


In [2]:
if not FACE_CASCADE_PATH.exists():
    raise FileNotFoundError(
        f"Face cascade not found at {FACE_CASCADE_PATH}. "
        "Place 'haarcascade_frontalface_default.xml' next to this notebook."
    )
if not EYE_CASCADE_PATH.exists():
    raise FileNotFoundError(
        f"Eye cascade not found at {EYE_CASCADE_PATH}. "
        "Place 'haarcascade_eye.xml' next to this notebook."
    )

face_cascade = cv2.CascadeClassifier(str(FACE_CASCADE_PATH))
eye_cascade  = cv2.CascadeClassifier(str(EYE_CASCADE_PATH))

if face_cascade.empty():
    raise RuntimeError("Failed to load face cascade. File may be corrupted.")
if eye_cascade.empty():
    raise RuntimeError("Failed to load eye cascade. File may be corrupted.")

print("Cascades loaded successfully.")


Cascades loaded successfully.


In [3]:
def detect_faces_and_eyes(frame, min_face=80, min_eye=20):
    """
    Detect faces and eyes on a BGR frame using Viola–Jones Haar cascades.
    Returns:
      faces: list of (x, y, w, h)
      eyes_per_face: list of lists, each inner list = eye rects relative to a face ROI
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # stabilize illumination for better detection

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(min_face, min_face),
    )

    eyes_per_face = []
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(
            roi_gray,
            scaleFactor=1.1,
            minNeighbors=6,
            minSize=(min_eye, min_eye),
        )
        eyes_per_face.append(eyes)

    return faces, eyes_per_face


def draw_annotations(frame, faces, eyes_per_face):
    """Draw rectangles: faces in green, eyes in blue."""
    for i, (x, y, w, h) in enumerate(faces):
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        for (ex, ey, ew, eh) in eyes_per_face[i]:
            cv2.rectangle(frame, (x + ex, y + ey), (x + ex + ew, y + ey + eh), (255, 0, 0), 2)
    return frame


def save_crops(frame, faces, eyes_per_face, save_faces=True, save_eyes=True, prefix="sample"):
    """
    Save cropped face and eye images into dataset directories.
    Returns: (num_face_crops_saved, num_eye_crops_saved)
    """
    ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    f_count, e_count = 0, 0

    for i, (x, y, w, h) in enumerate(faces):
        face_crop = frame[y:y+h, x:x+w]
        if save_faces and face_crop.size:
            out_path = FACE_OUT / f"{prefix}_face_{ts}_{i}.png"
            cv2.imwrite(str(out_path), face_crop)
            f_count += 1

        for j, (ex, ey, ew, eh) in enumerate(eyes_per_face[i]):
            eye_crop = frame[y+ey:y+ey+eh, x+ex:x+ex+ew]
            if save_eyes and eye_crop.size:
                out_path = EYE_OUT / f"{prefix}_eye_{ts}_{i}_{j}.png"
                cv2.imwrite(str(out_path), eye_crop)
                e_count += 1

    return f_count, e_count


In [4]:
cam_index = 0            
save_every_n_frames = 5  # Save crops every N frames while saving is enabled
max_saves = None         # e.g., set to an int to stop after fixed number of saved crops; None = unlimited

cap = cv2.VideoCapture(cam_index)
if not cap.isOpened():
    raise RuntimeError(f"Could not open camera index {cam_index}. Try a different index or check OS privacy permissions.")

print("Camera opened. Press 'q' to quit.")
saving_enabled = False
paused = False
frame_idx = 0
total_faces_saved = 0
total_eyes_saved = 0

try:
    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            faces, eyes_per_face = detect_faces_and_eyes(frame)

            # Draw overlays
            frame = draw_annotations(frame, faces, eyes_per_face)

            # Auto-save crops at interval
            if saving_enabled and (frame_idx % save_every_n_frames == 0):
                f_c, e_c = save_crops(frame, faces, eyes_per_face, prefix="rt")
                total_faces_saved += f_c
                total_eyes_saved += e_c

            # HUD text
            status = (f"[s]ave:{'ON' if saving_enabled else 'OFF'}  "
                      f"faces_saved:{total_faces_saved}  "
                      f"eyes_saved:{total_eyes_saved}  "
                      f"[p]ause:{'ON' if paused else 'OFF'}")
            cv2.putText(frame, status, (10, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (50, 220, 50), 2, cv2.LINE_AA)

            cv2.imshow("Real-Time Face & Eye Detection (q to quit)", frame)
            frame_idx += 1

        # Key handling (outside 'paused' so you can resume/quit)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            saving_enabled = not saving_enabled
        elif key == ord('p'):
            paused = not paused
        elif key == ord('c'):
            total_faces_saved = 0
            total_eyes_saved  = 0

        # Optional: stop after a fixed number of saves
        if max_saves is not None and (total_faces_saved + total_eyes_saved) >= max_saves:
            print("Reached max_saves limit; stopping.")
            break

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Camera released & windows closed.")


Camera opened. Press 'q' to quit.
Camera released & windows closed.
